In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime


import xgboost as xgb

# For tuning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from time import time

# Keras
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error


# loocv
from sklearn.datasets import make_blobs
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#import osgeo as gdal
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import geopandas as gpd
import rasterio as rio
import shutil, os
import shapely
import csv
import os
from rasterio.plot import show
from rasterio.warp import calculate_default_transform, reproject, Resampling

from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
from matplotlib import pyplot
import matplotlib
import matplotlib.pyplot as plt
import hydroeval as he

In [ ]:
ASO_files = ['CE20160401Data_set_SWE.csv', 'LB20160614_Data_set_SWE.csv', 'SJ20180304_Data_set_SWE.csv',
            'CE20170816Data_set_SWE.csv', 'TB20130403_Data_set_SWE.csv', 'MB20180425_Data_set_SWE.csv', 'SF20180423_Data_set_SWE.csv',
            'KC20190608_Data_set_SWE.csv', 'CE20160407Data_set_SWE.csv','LB20160621_Data_set_SWE.csv', 'TE20190503_Data_set_SWE.csv',
            'TB20130429_Data_set_SWE.csv', 'SJ20180422_Data_set_SWE.csv', 'LB20190715_Data_set_SWE.csv', 'TB20160527_Data_set_SWE.csv',
            'CE20160416Data_set_SWE.csv', 'LB20160626_Data_set_SWE.csv', 'KN20190417_Data_set_SWE.csv', 'SJ20190704_Data_set_SWE.csv',
            'TB20130525_Data_set_SWE.csv', 'TB20130503_Data_set_SWE.csv', 'LB20190703_Data_set_SWE.csv', 'SF20190502_Data_set_SWE.csv',
            'CE20160426Data_set_SWE.csv', 'TB20160708_Data_set_SWE.csv', 'LB20190501_Data_set_SWE.csv', 'SJ20180601_Data_set_SWE.csv',
            'TB20170727_Data_set_SWE.csv', 'MB20190716_Data_set_SWE.csv', 'TB20170129_Data_set_SWE.csv',
            'CE20160509Data_set_SWE.csv', 'SF20190609_Data_set_SWE.csv', 'TB20170717_Data_set_SWE.csv','TE20190705_Data_set_SWE.csv',
            'SJ20190325_Data_set_SWE.csv', 'TB20160509_Data_set_SWE.csv', 'LV20170717_Data_set_SWE.csv', 'MB20190703_Data_set_SWE.csv',
            'CE20170129Data_set_SWE.csv', 'KC20190427_Data_set_SWE.csv', 'TB20170816_Data_set_SWE.csv', 'LB20170718_Data_set_SWE.csv',
            'SF20190704_Data_set_SWE.csv', 'SJ20190713_Data_set_SWE.csv', 'LB20170128_Data_set_SWE.csv', 'TE20190417_Data_set_SWE.csv',
            'CE20170717Data_set_SWE.csv', 'TB20160426_Data_set_SWE.csv', 'LB20170815_Data_set_SWE.csv', 'KN20190611_Data_set_SWE.csv',
            'TB20160416_Data_set_SWE.csv', 'SJ20190501_Data_set_SWE.csv', 'RC20170717_Data_set_SWE.csv', 'SF20190317_Data_set_SWE.csv',
            'CE20170727Data_set_SWE.csv', 'TB20180528_Data_set_SWE.csv', 'TE20190613_Data_set_SWE.csv',
            'TB20160407_Data_set_SWE.csv', 'SJ20170815_Data_set_SWE.csv', 'MB20190604_Data_set_SWE.csv', 'LB20190611_Data_set_SWE.csv',
            'TB20160401_Data_set_SWE.csv', 'KN20180426_Data_set_SWE.csv', 'LB20190309_Data_set_SWE.csv', 'TB20180423_Data_set_SWE.csv',
            'SF20180601_Data_set_SWE.csv', 'TB20160326_Data_set_SWE.csv', 'SJ20190614_Data_set_SWE.csv', 'SF20190714_Data_set_SWE.csv',
            'CE20180423Data_set_SWE.csv', 'TB20130608_Data_set_SWE.csv', 'LB20180601_Data_set_SWE.csv', 'TE20190324_Data_set_SWE.csv',
            'TB20130601_Data_set_SWE.csv', 'SJ20170719_Data_set_SWE.csv', 'KC20190326_Data_set_SWE.csv', 'SF20170718_Data_set_SWE.csv',
            'CE20180528Data_set_SWE.csv', 'MB20190329_Data_set_SWE.csv', 'LB20180422_Data_set_S']

In [ ]:
def Hyperparameter_tuning(Training_files, Test_files, Validation_files, NUM_EPOCHS, saving_file_path):

  '''
  This function performs a bayesian hyperparameter optimization using the Training_files and Validation_files.
  It tests the models with the optimal 'best_parameters' on the Test_files.
  It saves the model in saving_file_path along with the a true vs predicted scatter plot and the training learning curves.
  It returns best_parameters, nse (R^2), and pearson correlation values on the Test_files.
  '''

    dataframe_train = pandas.read_csv(Training_files[0], index_col = 0 )

    for file in Training_files[1 :]:
        df_train= pandas.read_csv(file, index_col = 0 )
        dataframe_train = dataframe_train.append(df_train, ignore_index = True)

    X_train = dataframe_train.values[:, 2:9]
    y_train = dataframe_train.values[:, 9]

    #Validation set
    dataframe_val= pandas.read_csv(Validation_files[0], index_col = 0 )

    for file in Validation_files[1 :]:
        df_val= pandas.read_csv(file, index_col = 0 )
        dataframe_val = dataframe_val.append(df_val, ignore_index = True)

    X_val = dataframe_val.values[:, 2:9]
    y_val = dataframe_val.values[:, 9]

    # Testing dataset
    dataframe_test= pandas.read_csv(Test_files[0], index_col = 0 )

    for file in Test_files[1 :]:
        df_test= pandas.read_csv(file, index_col = 0 )
        dataframe_test = dataframe_test.append(df_test, ignore_index = True)

    X_test = dataframe_test.values[:, 2:9]
    y_test = dataframe_test.values[:, 9]


    #3: Need to scale the features for neural networks, otherwise the training doesn't converge.
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled =  scaler.transform(X_test)

    # This returns a multi-layer-perceptron model in Keras.
    def get_keras_model(num_hidden_layers,
                        num_neurons_per_layer,
                        dropout_rate,
                        activation):
        # create the MLP model.

        # define the layers.
        inputs = tf.keras.Input(shape=(X_train_scaled.shape[1],))  # input layer.
        x = layers.Dropout(dropout_rate)(inputs) # dropout on the weights.

        # Add the hidden layers.
        for i in range(num_hidden_layers):
            x = layers.Dense(num_neurons_per_layer,
                             activation=activation)(x)
            x = layers.Dropout(dropout_rate)(x)

        # output layer.
        outputs = layers.Dense(1, activation='linear')(x)

        model = tf.keras.Model(inputs=inputs, outputs=outputs)
        return model


    # This function takes in the hyperparameters and returns a score (Cross validation).
    def keras_mlp_cv_score(parameterization, weight=None):

        model = get_keras_model(parameterization.get('num_hidden_layers'),
                                parameterization.get('neurons_per_layer'),
                                parameterization.get('dropout_rate'),
                                parameterization.get('activation'))

        opt = parameterization.get('optimizer')
        opt = opt.lower()

        learning_rate = parameterization.get('learning_rate')

        if opt == 'adam':
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif opt == 'rms':
            optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        else:
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)


        # Specify the training configuration.
        model.compile(optimizer=optimizer,
                      loss=tf.keras.losses.MeanSquaredError(),
                      metrics=['mse'])

        data = X_train_scaled
        labels = y_train

        # fit the model using a the cross validation set.
        res = model.fit(data, labels, epochs=NUM_EPOCHS, batch_size=parameterization.get('batch_size'), validation_data = (X_val_scaled, y_val))

        # look at the last 10 epochs. Get the mean and standard deviation of the validation score.
        last10_scores = np.array(res.history['val_loss'][-10:])
        mean = last10_scores.mean()
        sem = last10_scores.std()

        # If the model didn't converge then set a high loss.
        if np.isnan(mean):
            return 9999.0, 0.0

        return mean, sem

    # Define the search space.
    parameters=[
        {
            "name": "learning_rate",
            "type": "range",
            "bounds": [0.0001, 0.5],
            "log_scale": True,
        },
        {
            "name": "dropout_rate",
            "type": "range",
            "bounds": [0.0001, 0.5],
            "log_scale": True,
        },
        {
            "name": "num_hidden_layers",
            "type": "range",
            "bounds": [7, 30],
            "value_type": "int"
        },
        {
            "name": "neurons_per_layer",
            "type": "range",
            "bounds": [40, 300],
            "value_type": "int"
        },
        {
            "name": "batch_size",
            "type": "choice",
            "values": [8, 16, 32, 64, 128, 256, 512],
        },

        {
            "name": "activation",
            "type": "choice",
            "values": ['tanh', 'sigmoid', 'relu'],
        },
        {
            "name": "optimizer",
            "type": "choice",
            "values": ['adam', 'rms', 'sgd'],
        },
    ]

    # import more packages
    from ax.service.ax_client import AxClient
    from ax.utils.notebook.plotting import render, init_notebook_plotting
    from ax.modelbridge.generation_strategy import GenerationStrategy, GenerationStep
    from ax.modelbridge.registry import Models, ModelRegistryBase
    from ax.modelbridge.dispatch_utils import choose_generation_strategy
    from ax.modelbridge.modelbridge_utils import get_pending_observation_features

    from ax.utils.testing.core_stubs import get_branin_search_space, get_branin_experiment
    init_notebook_plotting()

    ax_client = AxClient()

    # create the experiment.
    ax_client.create_experiment(
        name="keras_experiment",
        parameters=parameters,
        objective_name='keras_cv',
        minimize=True)

    def evaluate(parameters):
        return {"keras_cv": keras_mlp_cv_score(parameters)}



    # Based on the search space, this will use Sobol (https://en.wikipedia.org/wiki/Sobol_sequence) instead of a Guassian Process (GPEI).
    #
    # From the source code:
    #     """We should use only Sobol and not GPEI if:
    #     1. there are less continuous parameters in the search space than the sum of
    #     options for the choice parameters,
    #     2. the number of total iterations in the optimization is known in advance and
    #     there are less distinct points in the search space than the known intended
    #     number of total iterations.
    #     """

    # Sobol sequences seem to be better than random for high-dimensional spaces:
    # https://en.wikipedia.org/wiki/Quasi-Monte_Carlo_method

    for i in range(35):
        parameters, trial_index = ax_client.get_next_trial()
        ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))

    # look at all the trials.
    ax_client.get_trials_data_frame().sort_values('trial_index')

    best_parameters, values = ax_client.get_best_parameters()

    # the best set of parameters.
    for k in best_parameters.items():
        print(k)

    print()

    # the best score achieved.
    means, covariances = values
    print(means)

    keras_model = get_keras_model(best_parameters['num_hidden_layers'],
                              best_parameters['neurons_per_layer'],
                              best_parameters['dropout_rate'],
                              best_parameters['activation'])

    opt = best_parameters['optimizer']
    opt = opt.lower()

    learning_rate = best_parameters['learning_rate']

    if opt == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif opt == 'rms':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    # Specify the training configuration.
    keras_model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=['mse'])

    data = X_train_scaled
    labels = y_train
    res = keras_model.fit(data, labels, validation_data = (X_val_scaled, y_val), epochs=NUM_EPOCHS, batch_size=best_parameters['batch_size'])


    keras_model.save( filepath = saving_file_path )

    # Plot learning curve
    fig = matplotlib.pyplot.figure()
    plt.plot(res.history['loss'])
    plt.plot(res.history['val_loss'])
    plt.title('model loss on basin ')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

    fig.savefig( saving_file_path  +'/BM_LearningCurve.pdf')

    yhat = keras_model.predict(X_test_scaled)
    print(mean_squared_error(y_test, yhat))
    fig = matplotlib.pyplot.figure()
    plt.plot(y_test, label = 'observed')
    plt.plot(yhat, label = 'predicted')
    plt.xlabel('data point number')
    plt.ylabel('SWE (m)')
    plt.legend()
    fig.savefig(saving_file_path + '/BM_plot.pdf')

    fig = matplotlib.pyplot.figure()
    pyplot.scatter(y_test, yhat, c=dataframe_test['DEM'], cmap='terrain', vmin=2500, vmax=4100)
    plt.xlabel('SWE observed (m)')
    plt.ylabel('SWE predicted (m)')
    fig.savefig(saving_file_path  + '/BM_scatter.pdf')
    pyplot.show()
    # calculate the Pearson's correlation between two variables
    # seed random number generator
    seed(1)
    corr, _ = pearsonr(y_test, yhat.flatten())
    print('Pearsons correlation: %.3f' % corr)

    #Calculate NSE coefficient
    nse = he.evaluator(he.nse, yhat, y_test)
    print('NSE: %.3f' % nse)


    return nse, corr, best_parameters


In [ ]:
# For Base model 3 ( which is used in transfer learning)
Test_files = ASO_files[65: 80]
Training_files = ASO_files[0: 26] + ASO_files[39: 65]# 50 files
Validation_files = ASO_files[26: 39] # 15 files
saving_file_path '/home/users/lie08/California_base_models/Base_Model_3'
NUM_EPOCHS = 700
nse, corr, best_parameters= Hyperparameter_tuning(Training_files, Test_files, Validation_files, NUM_EPOCHS, saving_file_path)
print(nse)
print(corr)
print(best_parameters)
# load csv module
import csv

# define a dictionary with key value pairs
dict = {}
dict['NSE'] = nse
dict['R'] = corr
dict['learning_rate'] = best_parameters['learning_rate']
dict['dropout_rate'] = best_parameters['dropout_rate']
dict['num_hidden_layers'] = best_parameters['num_hidden_layers']
dict['neurons_per_layer'] = best_parameters['neurons_per_layer']
dict['batch_size'] = best_parameters['batch_size']
dict['activation'] = best_parameters['activation']
dict['optimizer'] = best_parameters['optimizer']
print(dict)

# open file for writing, "w" is writing
csvout = open("BM3.csv", "w")
w = csv.writer(csvout)

# loop over dictionary keys and values
for key, val in dict.items():

    # write every key and value to file
    w.writerow([key, val])
csvout.close()
